To do
* Revise style/adjective and artist list
* Create indicator variables

Importing libraries

In [ ]:
import numpy as np
import pandas as pd

# Webscraping

## Installing libraries

In [ ]:
pip install praw

In [ ]:
import praw

In [ ]:
cred = np.genfromtxt('cred.txt',dtype=str,delimiter=',')

Creating a Reddit Instance (need to hide the details in the cell later at some point)

In [ ]:
reddit = praw.Reddit(client_id=cred[0], client_secret=cred[1], user_agent=cred[2])

Let's play around with this

In [ ]:
hot_posts = reddit.subreddit('StableDiffusion').hot(limit=100)
for post in hot_posts:
  print(post.id)

In [ ]:
submission = reddit.submission(id='xcq819')
for y in submission.comments:
  print(y.body)

In [ ]:
type(submission.comments[0].body)

Hmmm after observing the "Prompt Included" posts, it seems that users are putting the prompts in the following places:


*   title
*   in a body of text where the images are linked elsewhere
*   comments

We're going to need to account for all of this later down the road



In [ ]:
posts = []
sd_subreddit = reddit.subreddit('StableDiffusion')
for post in sd_subreddit.hot(limit=1000):
  if post.link_flair_text == 'Prompt Included':
    posts.append([post.title,post.comments,post.url,post.id,post.score,post.num_comments,post.over_18])
    
posts = pd.DataFrame(posts,columns=['title',
                                    'comments',
                                    'url',
                                    'id',
                                    'score',
                                    'num_comments',
                                    'nsfw'])

In [ ]:
posts.head()

,title,comments,url,id,score,num_comments,nsfw
0,'This looks like a nice place for a nap.',"(iqzlmuh, ir0l1pa, ir0sc0q)",https://i.redd.it/if84ow9nnqr91.png,xv86kw,85,7,False
1,Just some cute hand sewn pumpkins,"(ir0x84g, ir190zu)",https://www.reddit.com/gallery/xvhq8v,xvhq8v,11,2,False
2,How to put HEAVY BlingBling golden chains arou...,"(iqz85k2, iqzhgiq, iqzs27l, iqzwn3g)",https://www.reddit.com/gallery/xv5bbd,xv5bbd,63,10,False
3,Dragonball Sesamestreet mash-up,(ir0euko),https://i.redd.it/v1hpyux5rqr91.jpg,xv8hcz,33,2,False
4,Christoph Waltz as a high ranking Imperial off...,(),https://i.redd.it/iha29x8vgtr91.jpg,xvjrpd,6,0,False


In [ ]:
posts.shape

(162, 7)

In [ ]:
comments = []
for i in posts.id:
  submission = reddit.submission(id=i)
  for j in submission.comments:
    comments.append([j.body,i])
comments = pd.DataFrame(comments,columns=['text','id'])  


In [ ]:
comments.iloc[0,0]

'Created with Stable Diffusion using the NMKD Stable Diffusion GUI - AI Image Generator.\r  \n\r  \nPrompt:\r  \nfemale ent, insanely detailed and intricate, hypermaximalist, elegant, ornate, hyper realistic, super detailed,, 8K, vivid colors, sunrise, open sky, black bear\r  \n\\-1029314660 -scale 7.50 -k euler a  \n\n\nYes, the female ent is supposed to be in there somewhere, but I never found her either.'

In [ ]:
comments.to_csv('/content/comments.csv')

It looks like comments are being cut off once converted to a csv

In [ ]:
comments.to_excel('/content/comments.xlsx')

Okay it looks like the whole comment is intact if you leave it in an Excel file format.

Okay so it looks like to get past the 1000 limit we can use PSAW (Pushshift) and get an index of id and then use PRAW to look at those posts and comments more in-depth. So let's figure out how the API works:

In [ ]:
pip install psaw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Creating the raw dataset

In [ ]:
from psaw import PushshiftAPI

In [ ]:
api = PushshiftAPI(reddit)

In [ ]:
#Grab a list of the ids of all posts from the Stable Diffusion subreddit
submissionList = list(api.search_submissions(subreddit='StableDiffusion'))

# Create temporary list
possiblePromptPost = []

# Get posts with "Prompt Included" flair
for post in submissionList:
  if post.link_flair_text == 'Prompt Included':
    possiblePromptPost.append(post)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

In [ ]:
temp = []
# Iterate through each post from the list of post ids obtained with PSAW
for submission in possiblePromptPost:

  # Using PRAW to create a submission object to get the text content
  post = reddit.submission(id = submission)

  # Add the text as a list
  temp.append([post.title,post.id,post.selftext,post.comments])
df = pd.DataFrame(temp, columns = ['title','id','text','comments'])
df

Streaming output truncated to the last 5000 lines.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.r

,title,id,text,comments
0,Man needlessly standing on roof overlooking cy...,xvrn98,,(ir2nbtb)
1,"first attempt at art nouveau prompts, in comments",xvrbxa,,(ir2l71a)
2,Lovecraftian Behemoth,xvqu7g,,(ir2imtj)
3,Henry Cavill medieval times(?,xvqpvg,,(ir2hh2s)
4,Android Pippi Longstocking in Neo Tokyo,xvpdis,,(ir2agth)
...,...,...,...,...
2385,Renaissance art of Keanu Reeves,woy7hy,,"(ikdgwmu, ikdl3yb, ikhe3c9)"
2386,Jazz robots.,wo3wm3,,"(ik8r5th, ik8qvj5, ikal2cb, ikcoao7, ikhgxtj)"
2387,Raves celebrating the day of the dead,wnei33,,"(ik4jkze, ik5eqzu)"
2388,Joker artwork,wm97qi,,"(ijxy1k4, ijzqeys)"


In [ ]:
# Export the dataframe to an excel file
df.to_excel('/content/prompt_data.xlsx')

## Adding text to the "comments" column

Up to this point, the "comments" column only has ids of the comments so we're going to have to iterate through them and add them.

In [ ]:
df = pd.read_excel('/content/prompt_data.xlsx')

In [ ]:
df.head()

,Unnamed: 0,title,id,text,comments
0,0,Man needlessly standing on roof overlooking cy...,xvrn98,NaN,<praw.models.comment_forest.CommentForest obje...
1,1,"first attempt at art nouveau prompts, in comments",xvrbxa,NaN,<praw.models.comment_forest.CommentForest obje...
2,2,Lovecraftian Behemoth,xvqu7g,NaN,<praw.models.comment_forest.CommentForest obje...
3,3,Henry Cavill medieval times(?,xvqpvg,NaN,<praw.models.comment_forest.CommentForest obje...
4,4,Android Pippi Longstocking in Neo Tokyo,xvpdis,NaN,<praw.models.comment_forest.CommentForest obje...


In [ ]:
print(df.comments[0])

We want the OP's comment since that's usually where the prompt is if they have decided to include the prompt in the comments.

In [ ]:
#Getting the OP's comments from the posts if any

#Create temporary list
temp = []

#Iterate through the first 50 posts
for id in df.id:

  #Create a submission object with the id
  submission = reddit.submission(id = id)

  #Check if the comment tree is empty
  if (len(submission.comments) == 0):
    temp.append("NA")
  else:
    # Create a temp list to store OP's comments' epoch times
    utc = []

    # Create a temp list to store OP's comments' ids
    op_comment_ids = []

    # Iterate through all the comments
    for comment in submission.comments:

      # Check if the comment is the OP
      if comment.is_submitter:

        # Add the epoch time to the list
        utc.append(comment.created_utc)

        # Add the comment's id to the op comment id list
        op_comment_ids.append(comment.id)

    for op_comment_id in op_comment_ids:
      curr_comment = reddit.comment(op_comment_id)
      # Check if the comment
      if curr_comment.created_utc == min(utc):

        # Append the comment to the temp list
        temp.append(curr_comment.body)
    
    # Append "NA" if there are no OP comments 
    if len(utc) == 0:
      temp.append("NA")
  
df.loc[:,'comments'] = temp
df.to_excel('/content/new_prompt_data.xlsx')


Streaming output truncated to the last 5000 lines.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.r

References:

[Reference used for webscraping Reddit](https://towardsdatascience.com/scraping-reddit-data-1c0af3040768)

[PRAW Documentation](https://praw.readthedocs.io/en/latest/index.html)

[Submission attributes](https://praw.readthedocs.io/en/latest/code_overview/models/submission.html)

[Getting posts with a specific flair](https://www.reddit.com/r/redditdev/comments/x4vydg/how_do_i_pull_posts_with_a_specific_flair_from_a/)

[Access more than 100 posts](https://www.reddit.com/r/redditdev/comments/t3j440/how_to_access_more_than_100_posts_with_praw/)

[How to Scrape Large Amounts of Reddit Data](https://medium.com/swlh/how-to-scrape-large-amounts-of-reddit-data-using-pushshift-1d33bde9286)

[Getting many/all submissions from a subreddit using PRAW/PSAW/pushshift](https://www.reddit.com/r/redditdev/comments/knbns9/getting_manyall_submissions_from_a_subreddit/)

[Using Pushshift API for data analysis on Reddit](https://medium.com/mcd-unison/using-pushshift-api-for-data-analysis-on-reddit-b08d339c48b8)

[Video: Reddit Scraping using PRAW and Pushshift](https://www.youtube.com/watch?v=a3zQdlLwEb4)

[Pushshift Reddit API v4.0 Documentation](https://reddit-api.readthedocs.io/en/latest/#search-parameters-for-comments)

[New to Pushshift? Read this! FAQ](https://www.reddit.com/r/pushshift/comments/bcxguf/new_to_pushshift_read_this_faq/)


